In [6]:
import pandas as pd
import numpy as np 
import os
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.metrics import mean_absolute_error, classification_report, accuracy_score
from xgboost import XGBClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [7]:
os.chdir("ETL Legislative 2T/data")

In [3]:
ls

cdsp_legi1997t2_circ.xlsx*    dataset_legislative_2nd_prediction_16_03.csv
cdsp_legi2002t2_circ.xls*     dataset_legislative_2nd_prediction.csv
cdsp_legi2007t2_circ.xls*     dico_variables.csv*
cdsp_legi2012t2_circ.xlsx*    pred_format_second_tour_du_6_6_21h46.xlsx
dataframe_elections.xlsx*     pred_format_second_tour_du_7_6_14h36.xlsx
dataset_legislative_2nd.csv*  pred_format_second_tour_du_7_6_15h35.xlsx


In [4]:
df = pd.read_csv("dataset_legislative_2nd_prediction_16_03.csv")
df["elu"] = df["elu"].map(lambda x: 1 if x=='O' else 0)

In [5]:
df.head()

,an,département,circonscription,code,name,first_name,bloc,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,...,top_triangulaire,bloc_opposant_Centre,bloc_opposant_Divers,bloc_opposant_Droite,bloc_opposant_Exdroite,bloc_opposant_Exgauche,bloc_opposant_Gauche,voix_bloc_1nd,voix_bloc_2nd_lag,voix_candidat_2nd_lag
0,2012,AIN,1.0,1|1,BRETON,XAVIER,Droite,1.0,1.0,0.2570,...,0,0,0,0,0,0,1,0.2241,0.567573,0.536621
1,2012,AIN,1.0,1|1,DEBAT,JEAN-FRANCOIS,Gauche,0.0,0.0,0.2147,...,0,0,0,1,0,0,0,0.2316,0.432427,0.463379
2,2012,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,Droite,1.0,1.0,0.2753,...,1,0,0,0,1,0,1,0.2209,0.567573,NaN
3,2012,AIN,2.0,1|2,EYRAUD,OLIVIER,Exdroite,0.0,0.0,0.1798,...,1,0,0,1,0,0,1,0.1252,NaN,NaN
4,2012,AIN,2.0,1|2,RAYMOND,MICHEL,Gauche,0.0,0.0,0.2027,...,1,0,0,1,1,0,0,0.1368,0.432427,NaN


In [6]:
df.columns

Index(['an', 'département', 'circonscription', 'code', 'name', 'first_name',
       'bloc', 'circo_leg_meme_nuance', 'circo_pres_meme_nuance',
       'score_bloc_pres', 'depute_sortant', 'ancien_depute', 'au_gouvernement',
       'ancien_ministre', 'p_voix', 'p_voix_candidat_2nd', 'elu',
       'membre_majorite', 'top_triangulaire', 'bloc_opposant_Centre',
       'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche', 'voix_bloc_1nd', 'voix_bloc_2nd_lag',
       'voix_candidat_2nd_lag'],
      dtype='object')

In [7]:
nd_tour = pd.get_dummies(pd.qcut(df["voix_bloc_2nd_lag"], 5, labels=False), prefix="bloc_2nd_tour" )
nd_tour["bloc_2nd_tour_nan"] = nd_tour.apply(lambda x : 0 if np.sum(x)==1 else 1 , axis=1)

nd_tour_candidate = pd.get_dummies(pd.qcut(df["voix_candidat_2nd_lag"], 5, labels=False), 
                                   prefix="2nd_tour_candidate" )
nd_tour_candidate["2nd_tour_candidate_nan"] = nd_tour_candidate.apply(lambda x : 0 if np.sum(x)==1 else 1 , axis=1)

nd = pd.concat([nd_tour, nd_tour_candidate], axis=1)

In [8]:
df = pd.concat([df, nd], axis=1)

In [9]:
df.columns

Index(['an', 'département', 'circonscription', 'code', 'name', 'first_name',
       'bloc', 'circo_leg_meme_nuance', 'circo_pres_meme_nuance',
       'score_bloc_pres', 'depute_sortant', 'ancien_depute', 'au_gouvernement',
       'ancien_ministre', 'p_voix', 'p_voix_candidat_2nd', 'elu',
       'membre_majorite', 'top_triangulaire', 'bloc_opposant_Centre',
       'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche', 'voix_bloc_1nd', 'voix_bloc_2nd_lag',
       'voix_candidat_2nd_lag', 'bloc_2nd_tour_0.0', 'bloc_2nd_tour_1.0',
       'bloc_2nd_tour_2.0', 'bloc_2nd_tour_3.0', 'bloc_2nd_tour_4.0',
       'bloc_2nd_tour_nan', '2nd_tour_candidate_0.0', '2nd_tour_candidate_1.0',
       '2nd_tour_candidate_2.0', '2nd_tour_candidate_3.0',
       '2nd_tour_candidate_4.0', '2nd_tour_candidate_nan'],
      dtype='object')

In [10]:
del df['voix_candidat_2nd_lag']
del df['voix_bloc_2nd_lag']

In [11]:
df = pd.get_dummies(df, columns=["bloc"], prefix="bloc")

In [12]:
df.columns

Index(['an', 'département', 'circonscription', 'code', 'name', 'first_name',
       'circo_leg_meme_nuance', 'circo_pres_meme_nuance', 'score_bloc_pres',
       'depute_sortant', 'ancien_depute', 'au_gouvernement', 'ancien_ministre',
       'p_voix', 'p_voix_candidat_2nd', 'elu', 'membre_majorite',
       'top_triangulaire', 'bloc_opposant_Centre', 'bloc_opposant_Divers',
       'bloc_opposant_Droite', 'bloc_opposant_Exdroite',
       'bloc_opposant_Exgauche', 'bloc_opposant_Gauche', 'voix_bloc_1nd',
       'bloc_2nd_tour_0.0', 'bloc_2nd_tour_1.0', 'bloc_2nd_tour_2.0',
       'bloc_2nd_tour_3.0', 'bloc_2nd_tour_4.0', 'bloc_2nd_tour_nan',
       '2nd_tour_candidate_0.0', '2nd_tour_candidate_1.0',
       '2nd_tour_candidate_2.0', '2nd_tour_candidate_3.0',
       '2nd_tour_candidate_4.0', '2nd_tour_candidate_nan', 'bloc_Centre',
       'bloc_Divers', 'bloc_Droite', 'bloc_Exdroite', 'bloc_Exgauche',
       'bloc_Gauche'],
      dtype='object')

In [13]:
df.head()

,an,département,circonscription,code,name,first_name,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,...,2nd_tour_candidate_2.0,2nd_tour_candidate_3.0,2nd_tour_candidate_4.0,2nd_tour_candidate_nan,bloc_Centre,bloc_Divers,bloc_Droite,bloc_Exdroite,bloc_Exgauche,bloc_Gauche
0,2012,AIN,1.0,1|1,BRETON,XAVIER,1.0,1.0,0.2570,1.0,...,1,0,0,0,0,0,1,0,0,0
1,2012,AIN,1.0,1|1,DEBAT,JEAN-FRANCOIS,0.0,0.0,0.2147,0.0,...,0,0,0,0,0,0,0,0,0,1
2,2012,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,1.0,1.0,0.2753,1.0,...,0,0,0,1,0,0,1,0,0,0
3,2012,AIN,2.0,1|2,EYRAUD,OLIVIER,0.0,0.0,0.1798,0.0,...,0,0,0,1,0,0,0,1,0,0
4,2012,AIN,2.0,1|2,RAYMOND,MICHEL,0.0,0.0,0.2027,0.0,...,0,0,0,1,0,0,0,0,0,1


In [14]:
df["score_pre_maj"] = df.apply(lambda x: x["score_bloc_pres"] if x['membre_majorite']==1 else 0, axis=1)

In [15]:
df = df[[
        'an', 'département','circonscription','code', 'name','first_name',
       'circo_leg_meme_nuance', 'circo_pres_meme_nuance', 
       'depute_sortant', 'ancien_depute', 'au_gouvernement', 'ancien_ministre','membre_majorite',"score_pre_maj",
        "score_bloc_pres",
    
        'bloc_opposant_Centre', 'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche',
        
        'bloc_2nd_tour_0.0',
       'bloc_2nd_tour_1.0', 'bloc_2nd_tour_2.0', 'bloc_2nd_tour_3.0',
       'bloc_2nd_tour_4.0', 'bloc_2nd_tour_nan', '2nd_tour_candidate_0.0',
       '2nd_tour_candidate_1.0', '2nd_tour_candidate_2.0',
       '2nd_tour_candidate_3.0', '2nd_tour_candidate_4.0',
       '2nd_tour_candidate_nan', 
    
        
        'bloc_Centre', 'bloc_Divers', 'bloc_Droite',
       'bloc_Exdroite', 'bloc_Exgauche', 'bloc_Gauche',
    
       'p_voix', 'voix_bloc_1nd', 'top_triangulaire', 'p_voix_candidat_2nd', 'elu']]
        


In [16]:
df.head()

,an,département,circonscription,code,name,first_name,circo_leg_meme_nuance,circo_pres_meme_nuance,depute_sortant,ancien_depute,...,bloc_Divers,bloc_Droite,bloc_Exdroite,bloc_Exgauche,bloc_Gauche,p_voix,voix_bloc_1nd,top_triangulaire,p_voix_candidat_2nd,elu
0,2012,AIN,1.0,1|1,BRETON,XAVIER,1.0,1.0,1.0,1.0,...,0,1,0,0,0,0.2241,0.2241,0,0.515715,1
1,2012,AIN,1.0,1|1,DEBAT,JEAN-FRANCOIS,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0.2316,0.2316,0,0.484285,0
2,2012,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,1.0,1.0,1.0,1.0,...,0,1,0,0,0,0.2209,0.2209,1,0.443146,1
3,2012,AIN,2.0,1|2,EYRAUD,OLIVIER,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0.1252,0.1252,1,0.169303,0
4,2012,AIN,2.0,1|2,RAYMOND,MICHEL,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0.1368,0.1368,1,0.387551,0


In [18]:
#X_train = df[(df["an"]!=2012)&(df["an"]!=1997)]
#X_test = df[df["an"]==2012]
#y_train = df.ix[(df["an"]!=2012)&(df["an"]!=1997), "score"]
#y_test = df.ix[df["an"]==2012, "score"]
#y_train_cl = df.ix[(df["an"]!=2012)&(df["an"]!=1997), "elu"]
#y_test_cl = df.ix[df["an"]==2012, "elu"]

In [18]:
#del X_train["score"]
#del X_test["score"]
#del X_train["elu"]
#del X_test["elu"]

In [19]:
#del X_train["an"]
#del X_test["code"]
#del X_train["code"]
#del X_test["an"]

In [20]:
#X_train.columns

In [21]:
#rl = LinearRegression()
#rl.fit(X_train[['voix_bloc_1nd',  'top_triangulaire']], y_train)

In [22]:
#rl.score(X_test[['voix_bloc_1nd',  'top_triangulaire']], y_test)

In [23]:
#rf = RandomForestRegressor()
#rf.fit(X_train, y_train)

In [24]:
#rf.score(X_test, y_test)

In [25]:
#svm = SVR(kernel="rbf",C=15, gamma=0.2)
#svm.fit(X_train, y_train)

In [26]:
#svm.score(X_test, y_test)

In [27]:
#mean_absolute_error(y_test, svm.predict(X_test))

In [24]:
X_train = df[(df["an"]!=2012)&(df["an"]!=1997)&(df["an"]!=2017)]
X_test = df[df["an"]==2012]
y_train_cl = df.ix[(df["an"]!=2012)&(df["an"]!=1997)&(df["an"]!=2017), "elu"]
y_test_cl = df.ix[df["an"]==2012, "elu"]
X_validation = df[(df["an"]==2017)]
del X_train['p_voix_candidat_2nd']
del X_test['p_voix_candidat_2nd']
del X_train["elu"]
del X_test["elu"]
del X_train["an"]
del X_test["an"]
del X_train["code"]
del X_train["département"]
del X_test["département"]
del X_train["name"]
del X_test["name"]
del X_train['first_name']
del X_test['first_name']
del X_train['circonscription']
del X_test['circonscription']

In [25]:
X_test = X_test[["code",'p_voix','score_pre_maj','bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
'top_triangulaire','ancien_ministre', 'au_gouvernement','circo_pres_meme_nuance']]

In [26]:
X_train = X_train[['p_voix','score_pre_maj','bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
'top_triangulaire','ancien_ministre', 'au_gouvernement','circo_pres_meme_nuance']]

In [29]:
clf1 = SVC(probability=True)
clf2 = RandomForestClassifier()
clf3= XGBClassifier(min_child_weight=2, n_estimators=400, 
                    learning_rate=0.02, max_depth=3)
clf4 = LogisticRegression()

clf1.fit(X_train, y_train_cl)
clf2.fit(X_train, y_train_cl)
clf3.fit(X_train, y_train_cl)
clf4.fit(X_train, y_train_cl)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [30]:
predict_1 = clf1.predict_proba(X_test.drop(["code"], axis=1))
predict_2 = clf2.predict_proba(X_test.drop(["code"], axis=1))
predict_3 = clf3.predict_proba(X_test.drop(["code"], axis=1))
predict_4 = clf4.predict_proba(X_test.drop(["code"], axis=1))

In [31]:
clf1.classes_

array([0, 1])

In [32]:
df_1 = pd.Series( (v[1] for v in predict_1) , name="svc")
df_2 = pd.Series( (v[1] for v in predict_2) , name="rf")
df_3 = pd.Series( (v[1] for v in predict_3) , name="xgb")
df_4 = pd.Series( (v[1] for v in predict_4) , name="lg")

In [33]:
X_test = X_test.reset_index(drop=True)

In [34]:
X_test = pd.concat([X_test, df_1, df_2, df_3,df_4], axis=1)

In [35]:
X_test

,code,p_voix,score_pre_maj,bloc_opposant_Exdroite,bloc_opposant_Exgauche,top_triangulaire,ancien_ministre,au_gouvernement,circo_pres_meme_nuance,svc,rf,xgb,lg
0,1|1,0.2241,0.0000,0,0,0,0.0,0.0,1.0,0.786552,1.000000,0.846050,0.776876
1,1|1,0.2316,0.2147,0,0,0,0.0,0.0,0.0,0.316085,0.600000,0.332751,0.386535
2,1|2,0.2209,0.0000,1,0,1,0.0,0.0,1.0,0.708659,0.900000,0.902178,0.892028
3,1|2,0.1252,0.0000,0,0,1,0.0,0.0,0.0,0.082744,0.100000,0.094368,0.056167
4,1|2,0.1368,0.2027,1,0,1,0.0,0.0,0.0,0.397533,0.600000,0.226275,0.387923
5,1|3,0.2214,0.0000,0,0,0,0.0,0.0,1.0,0.785891,1.000000,0.846050,0.772397
6,1|3,0.1742,0.2090,0,0,0,0.0,0.0,0.0,0.297863,0.100000,0.178198,0.268638
7,1|4,0.1823,0.1944,0,0,0,0.0,0.0,0.0,0.300881,0.000000,0.183250,0.287053
8,1|4,0.2370,0.0000,0,0,0,0.0,0.0,1.0,0.789688,1.000000,0.900251,0.797402
9,1|5,0.1795,0.0000,0,0,0,0.0,0.0,1.0,0.775430,0.200000,0.629466,0.695031


In [36]:
X_test["mean"] = X_test.apply(lambda x: np.max([x["svc"], x["xgb"], x["rf"]]), axis=1)
X_test["max"] = X_test.apply(lambda x: np.mean([x["svc"], x["xgb"], x["rf"]]), axis=1)

In [37]:
X_test.shape

(1101, 15)

In [38]:
X_test = pd.merge(X_test, X_test.groupby(["code"])["rf"].max().to_frame().reset_index(), how="left", on="code")
X_test = pd.merge(X_test, X_test.groupby(["code"])["svc"].max().to_frame().reset_index(), how="left", on="code")
X_test = pd.merge(X_test, X_test.groupby(["code"])["xgb"].max().to_frame().reset_index(), how="left", on="code")
X_test = pd.merge(X_test, X_test.groupby(["code"])["mean"].max().to_frame().reset_index(), how="left", on="code")
X_test = pd.merge(X_test, X_test.groupby(["code"])["max"].max().to_frame().reset_index(), how="left", on="code")
X_test = pd.merge(X_test, X_test.groupby(["code"])["lg"].max().to_frame().reset_index(), how="left", on="code")

In [39]:
X_test.head()

,code,p_voix,score_pre_maj,bloc_opposant_Exdroite,bloc_opposant_Exgauche,top_triangulaire,ancien_ministre,au_gouvernement,circo_pres_meme_nuance,svc_x,...,xgb_x,lg_x,mean_x,max_x,rf_y,svc_y,xgb_y,mean_y,max_y,lg_y
0,1|1,0.2241,0.0000,0,0,0,0.0,0.0,1.0,0.786552,...,0.846050,0.776876,1.000000,0.877534,1.0,0.786552,0.846050,1.000000,0.877534,0.776876
1,1|1,0.2316,0.2147,0,0,0,0.0,0.0,0.0,0.316085,...,0.332751,0.386535,0.600000,0.416279,1.0,0.786552,0.846050,1.000000,0.877534,0.776876
2,1|2,0.2209,0.0000,1,0,1,0.0,0.0,1.0,0.708659,...,0.902178,0.892028,0.902178,0.836946,0.9,0.708659,0.902178,0.902178,0.836946,0.892028
3,1|2,0.1252,0.0000,0,0,1,0.0,0.0,0.0,0.082744,...,0.094368,0.056167,0.100000,0.092371,0.9,0.708659,0.902178,0.902178,0.836946,0.892028
4,1|2,0.1368,0.2027,1,0,1,0.0,0.0,0.0,0.397533,...,0.226275,0.387923,0.600000,0.407936,0.9,0.708659,0.902178,0.902178,0.836946,0.892028


In [40]:
X_test["label_svc"] = X_test.apply(lambda x: 0 if x["svc_x"]<x["svc_y"] else 1, axis=1)
X_test["label_rf"] = X_test.apply(lambda x: 0 if x["rf_x"]<x["rf_y"] else 1, axis=1)
X_test["label_xgb"] = X_test.apply(lambda x: 0 if x["xgb_x"]<x["xgb_y"] else 1, axis=1)
X_test["label_mean"] = X_test.apply(lambda x: 0 if x["mean_x"]<x["mean_y"] else 1, axis=1)
X_test["label_max"] = X_test.apply(lambda x: 0 if x["max_x"]<x["max_y"] else 1, axis=1)
X_test["label_lg"] = X_test.apply(lambda x: 0 if x["lg_x"]<x["lg_y"] else 1, axis=1)

In [41]:
X_test

,code,p_voix,score_pre_maj,bloc_opposant_Exdroite,bloc_opposant_Exgauche,top_triangulaire,ancien_ministre,au_gouvernement,circo_pres_meme_nuance,svc_x,...,xgb_y,mean_y,max_y,lg_y,label_svc,label_rf,label_xgb,label_mean,label_max,label_lg
0,1|1,0.2241,0.0000,0,0,0,0.0,0.0,1.0,0.786552,...,0.846050,1.000000,0.877534,0.776876,1,1,1,1,1,1
1,1|1,0.2316,0.2147,0,0,0,0.0,0.0,0.0,0.316085,...,0.846050,1.000000,0.877534,0.776876,0,0,0,0,0,0
2,1|2,0.2209,0.0000,1,0,1,0.0,0.0,1.0,0.708659,...,0.902178,0.902178,0.836946,0.892028,1,1,1,1,1,1
3,1|2,0.1252,0.0000,0,0,1,0.0,0.0,0.0,0.082744,...,0.902178,0.902178,0.836946,0.892028,0,0,0,0,0,0
4,1|2,0.1368,0.2027,1,0,1,0.0,0.0,0.0,0.397533,...,0.902178,0.902178,0.836946,0.892028,0,0,0,0,0,0
5,1|3,0.2214,0.0000,0,0,0,0.0,0.0,1.0,0.785891,...,0.846050,1.000000,0.877314,0.772397,1,1,1,1,1,1
6,1|3,0.1742,0.2090,0,0,0,0.0,0.0,0.0,0.297863,...,0.846050,1.000000,0.877314,0.772397,0,0,0,0,0,0
7,1|4,0.1823,0.1944,0,0,0,0.0,0.0,0.0,0.300881,...,0.900251,1.000000,0.896646,0.797402,0,0,0,0,0,0
8,1|4,0.2370,0.0000,0,0,0,0.0,0.0,1.0,0.789688,...,0.900251,1.000000,0.896646,0.797402,1,1,1,1,1,1
9,1|5,0.1795,0.0000,0,0,0,0.0,0.0,1.0,0.775430,...,0.629466,0.775430,0.534965,0.695031,1,1,1,1,1,1


In [42]:
print(classification_report(X_test["label_svc"], y_test_cl))
print(accuracy_score(X_test["label_svc"], y_test_cl))

             precision    recall  f1-score   support

          0       0.79      0.79      0.79       560
          1       0.78      0.78      0.78       541

avg / total       0.79      0.79      0.79      1101

0.785649409628


In [43]:
print(classification_report(X_test["label_rf"], y_test_cl))
print(accuracy_score(X_test["label_rf"], y_test_cl))

             precision    recall  f1-score   support

          0       0.73      0.79      0.76       519
          1       0.79      0.74      0.77       582

avg / total       0.76      0.76      0.76      1101

0.761126248865


In [44]:
print(classification_report(X_test["label_xgb"], y_test_cl))
print(accuracy_score(X_test["label_xgb"], y_test_cl))

             precision    recall  f1-score   support

          0       0.79      0.79      0.79       559
          1       0.79      0.78      0.78       542

avg / total       0.79      0.79      0.79      1101

0.788374205268


In [45]:
print(classification_report(X_test["label_mean"], y_test_cl))
print(accuracy_score(X_test["label_mean"], y_test_cl))

             precision    recall  f1-score   support

          0       0.78      0.79      0.78       554
          1       0.78      0.77      0.78       547

avg / total       0.78      0.78      0.78      1101

0.780199818347


In [46]:
print(classification_report(X_test["label_max"], y_test_cl))
print(accuracy_score(X_test["label_max"], y_test_cl))

             precision    recall  f1-score   support

          0       0.80      0.80      0.80       560
          1       0.79      0.79      0.79       541

avg / total       0.80      0.80      0.80      1101

0.798365122616


In [47]:
print(classification_report(X_test["label_lg"], y_test_cl))
print(accuracy_score(X_test["label_lg"], y_test_cl))

             precision    recall  f1-score   support

          0       0.79      0.79      0.79       560
          1       0.79      0.79      0.79       541

avg / total       0.79      0.79      0.79      1101

0.789282470481


In [50]:
gbm = XGBClassifier()
gbm_params = {
'learning_rate': [0.01,0.1,0.3],
'n_estimators': [300,350,400,1000,1500],
'max_depth': [4,5,6,7],
"min_child_weight":[2,5,10,15]
}


X_train = X_train[['p_voix','voix_bloc_1nd','score_pre_maj', 'circo_pres_meme_nuance','circo_leg_meme_nuance',
'depute_sortant','ancien_depute','top_triangulaire','ancien_ministre', 'au_gouvernement']]
grid = GridSearchCV(gbm, gbm_params,verbose=10,n_jobs=-1)
grid.fit(X_train, y_train_cl)
print (grid.best_params_)

Fitting 3 folds for each of 240 candidates, totalling 720 fits
[CV] max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=2 
[CV] max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=2 
[CV] max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=2 
[CV] max_depth=4, n_estimators=350, learning_rate=0.01, min_child_weight=2 
[CV]  max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=2, score=0.787879 -   1.6s
[CV] max_depth=4, n_estimators=350, learning_rate=0.01, min_child_weight=2 
[CV]  max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=2, score=0.763636 -   1.7s
[CV] max_depth=4, n_estimators=350, learning_rate=0.01, min_child_weight=2 
[CV]  max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=2, score=0.784848 -   1.8s
[CV] max_depth=4, n_estimators=400, learning_rate=0.01, min_child_weight=2 
[CV]  max_depth=4, n_estimators=350, learning_rate=0.01, min_child_weight=2, score=0.793939 -   2.0s
[

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.7s


[CV]  max_depth=4, n_estimators=400, learning_rate=0.01, min_child_weight=2, score=0.795455 -   2.4s
[CV] max_depth=4, n_estimators=1000, learning_rate=0.01, min_child_weight=2 
[CV]  max_depth=4, n_estimators=400, learning_rate=0.01, min_child_weight=2, score=0.787879 -   2.3s
[CV] max_depth=4, n_estimators=1000, learning_rate=0.01, min_child_weight=2 
[CV]  max_depth=4, n_estimators=400, learning_rate=0.01, min_child_weight=2, score=0.763636 -   2.2s
[CV] max_depth=4, n_estimators=1500, learning_rate=0.01, min_child_weight=2 
[CV]  max_depth=4, n_estimators=1000, learning_rate=0.01, min_child_weight=2, score=0.762121 -   4.2s
[CV] max_depth=4, n_estimators=1500, learning_rate=0.01, min_child_weight=2 


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    8.7s


[CV]  max_depth=4, n_estimators=1000, learning_rate=0.01, min_child_weight=2, score=0.801515 -   5.0s
[CV] max_depth=4, n_estimators=1500, learning_rate=0.01, min_child_weight=2 
[CV]  max_depth=4, n_estimators=1000, learning_rate=0.01, min_child_weight=2, score=0.793939 -   5.0s
[CV] max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=5 
[CV]  max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=5, score=0.762121 -   1.6s
[CV] max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=5 
[CV]  max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=5, score=0.786364 -   1.6s
[CV] max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=5 
[CV]  max_depth=4, n_estimators=1500, learning_rate=0.01, min_child_weight=2, score=0.796970 -   7.6s
[CV] max_depth=4, n_estimators=350, learning_rate=0.01, min_child_weight=5 
[CV]  max_depth=4, n_estimators=300, learning_rate=0.01, min_child_weight=5, score=0.759091 -   1.4s
[CV] max_d

...

[CV]  max_depth=6, n_estimators=1000, learning_rate=0.3, min_child_weight=10, score=0.759091 -   7.3s
[CV] max_depth=6, n_estimators=1500, learning_rate=0.3, min_child_weight=10 
[CV]  max_depth=6, n_estimators=1000, learning_rate=0.3, min_child_weight=10, score=0.736364 -   7.5s
[CV] max_depth=6, n_estimators=300, learning_rate=0.3, min_child_weight=15 
[CV]  max_depth=6, n_estimators=300, learning_rate=0.3, min_child_weight=15, score=0.796970 -   2.4s
[CV] max_depth=6, n_estimators=300, learning_rate=0.3, min_child_weight=15 
[CV]  max_depth=6, n_estimators=1500, learning_rate=0.3, min_child_weight=10, score=0.760606 -  10.4s
[CV] max_depth=6, n_estimators=300, learning_rate=0.3, min_child_weight=15 
[CV]  max_depth=6, n_estimators=300, learning_rate=0.3, min_child_weight=15, score=0.775758 -   2.0s
[CV] max_depth=6, n_estimators=350, learning_rate=0.3, min_child_weight=15 
[CV]  max_depth=6, n_estimators=300, learning_rate=0.3, min_child_weight=15, score=0.765152 -   2.0s
[CV] max_d

[Parallel(n_jobs=-1)]: Done 677 tasks      | elapsed: 13.7min


[CV]  max_depth=7, n_estimators=350, learning_rate=0.3, min_child_weight=5, score=0.757576 -   2.3s
[CV] max_depth=7, n_estimators=400, learning_rate=0.3, min_child_weight=5 
[CV]  max_depth=7, n_estimators=1500, learning_rate=0.3, min_child_weight=2, score=0.733333 -  11.7s
[CV]  max_depth=7, n_estimators=1500, learning_rate=0.3, min_child_weight=2, score=0.745455 -  12.1s
[CV] max_depth=7, n_estimators=400, learning_rate=0.3, min_child_weight=5 
[CV] max_depth=7, n_estimators=400, learning_rate=0.3, min_child_weight=5 
[CV]  max_depth=7, n_estimators=350, learning_rate=0.3, min_child_weight=5, score=0.754545 -   3.6s
[CV] max_depth=7, n_estimators=1000, learning_rate=0.3, min_child_weight=5 
[CV]  max_depth=7, n_estimators=400, learning_rate=0.3, min_child_weight=5, score=0.771212 -   3.4s
[CV] max_depth=7, n_estimators=1000, learning_rate=0.3, min_child_weight=5 
[CV]  max_depth=7, n_estimators=400, learning_rate=0.3, min_child_weight=5, score=0.754545 -   2.9s
[CV] max_depth=7, n_e

[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 14.7min finished


{'max_depth': 6, 'n_estimators': 1500, 'learning_rate': 0.01, 'min_child_weight': 10}


In [56]:
print(grid.best_score_)

0.7929292929292929


In [202]:
X_test.columns

Index(['circo_leg_meme_nuance', 'code', 'circo_pres_meme_nuance',
       'score_bloc_pres', 'depute_sortant', 'ancien_depute', 'au_gouvernement',
       'ancien_ministre', 'top_triangulaire', 'bloc_opposant_Centre',
       'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche', 'voix_bloc_1nd', 'bloc_Centre', 'bloc_Divers',
       'bloc_Droite', 'bloc_Exdroite', 'bloc_Exgauche', 'bloc_Gauche', 'svc_x',
       'rf_x', 'xgb_x', 'mean_x', 'max_x', 'rf_y', 'svc_y', 'xgb_y', 'mean_y',
       'max_y', 'label_svc', 'label_rf', 'label_xgb', 'label_mean',
       'label_max'],
      dtype='object')

In [49]:
clf_f = LogisticRegression()


X_train = df[(df["an"]!=1997)&(df["an"]!=2017)]
y_train_cl = df.ix[(df["an"]!=1997)&(df["an"]!=2017), "elu"]
X_validation = df[(df["an"]==2017)]
del X_train['p_voix_candidat_2nd']
del X_train["elu"]
del X_train["an"]
del X_train["département"]
del X_train["code"]
del X_train["name"]
del X_train["first_name"]
del X_train['circonscription']

del X_validation["an"]
del X_validation["p_voix_candidat_2nd"]
del X_validation['elu']

In [50]:
X_train = X_train[['p_voix','score_pre_maj','bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
'top_triangulaire','ancien_ministre', 'au_gouvernement','circo_pres_meme_nuance']]

In [51]:
#cl = DecisionTreeClassifier()
#cl.fit(X_train, y_train_cl)
#sorted(list(zip(cl.feature_importances_, X_train.columns)), reverse=True)

In [52]:
X_train.columns

Index(['p_voix', 'score_pre_maj', 'bloc_opposant_Exdroite',
       'bloc_opposant_Exgauche', 'top_triangulaire', 'ancien_ministre',
       'au_gouvernement', 'circo_pres_meme_nuance'],
      dtype='object')

In [53]:
X_validation = X_validation[['département', 'circonscription', 'code', 'name', 'first_name',
    'p_voix','score_pre_maj','bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
'top_triangulaire','ancien_ministre', 'au_gouvernement','circo_pres_meme_nuance', ]]

In [54]:
clf_f.fit(X_train, y_train_cl)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [55]:
predict = clf_f.predict_proba(X_validation.drop(["département", 'circonscription',"code","name","first_name"], axis=1))

In [56]:
df_prob = pd.Series( (v[1] for v in predict) , name="predict_proba")
X_validation = X_validation.reset_index(drop=True)
X_validation = pd.concat([X_validation, df_prob], axis=1)

In [57]:
X_validation = pd.merge(X_validation, X_validation.groupby(["code"])["predict_proba"].max().to_frame().reset_index(), how="left", on="code")

In [58]:
X_validation["elu"] = X_validation.apply(lambda x: 0 if x["predict_proba_x"]<x["predict_proba_y"] else 1, axis=1)

In [59]:
te = X_validation.groupby(["code"])["elu"].sum().reset_index()
te[te["elu"]>1]

,code,elu
561,987|3,2


In [61]:
X_validation[X_validation["code"]=="987|3"]

,département,circonscription,code,name,first_name,p_voix,score_pre_maj,bloc_opposant_Exdroite,bloc_opposant_Exgauche,top_triangulaire,ancien_ministre,au_gouvernement,circo_pres_meme_nuance,predict_proba_x,predict_proba_y,elu
841,POLYNESIE-FRANCAISE,3.0,987|3,DUBOIS,VINCENT,0.202497,0.0,0,0,0,0.0,0.0,0.0,0.352509,0.352509,1
842,POLYNESIE-FRANCAISE,3.0,987|3,HOWELL,PATRICK,0.202497,0.0,0,0,0,0.0,0.0,0.0,0.352509,0.352509,1


In [62]:
df_col = pd.read_excel("dataframe_elections.xlsx")
df_col = df_col[df_col["an"]==2017]
df_col  = df_col.rename(columns={"nom":"name","prenom":"first_name"})
df_col.head()

,an,c_dep,dep,circo,code,inscrits,geo_frontalier,geo_dom,geo_idf,etrangers,...,sexe,name,first_name,etiquette,nuance,nuance_groupe,bloc,voix,p_voix,second_tour
0,2017,1,AIN,1,1|1,82653.0,1,0,0,0.0793,...,F,BLATRIX-CONTAT,FLORENCE,NaN,SOC,SOC,Gauche,NaN,NaN,NaN
1,2017,1,AIN,1,1|1,82653.0,1,0,0,0.0793,...,M,BONNOT,GILBERT,NaN,DIV,DIV,Divers,NaN,NaN,NaN
2,2017,1,AIN,1,1|1,82653.0,1,0,0,0.0793,...,M,BRETON,XAVIER,NaN,LR,LR,Droite,NaN,NaN,NaN
3,2017,1,AIN,1,1|1,82653.0,1,0,0,0.0793,...,M,BUISSON,JEROME,NaN,FN,FN,Exdroite,NaN,NaN,NaN
4,2017,1,AIN,1,1|1,82653.0,1,0,0,0.0793,...,F,CARLIER,MARIE,NaN,DIV,DIV,Divers,NaN,NaN,NaN


In [63]:
# add partie politique
X_validation_df = pd.merge(X_validation, df_col[["code","name","first_name","nuance_groupe"]], 
                     on=["code","name","first_name"],how="left")

In [64]:
#list_circo = X_validation_df.ix[(X_validation_df["nuance_groupe"]=="SOC")&(X_validation_df["elu"]==1),"code"].tolist()
#X_validation_df[X_validation_df["code"].isin(list_circo)]

In [65]:
X_validation_df.groupby(["nuance_groupe"])["elu"].sum()

nuance_groupe
COM      3
DIV      2
DLF      0
DVD      4
DVG     14
ECO      5
EXD      0
FI      33
FN       5
LR     105
MDM     45
RDG      3
REM    278
SOC     64
UDI     17
Name: elu, dtype: int64

In [9]:
X_validation_df = pd.read_csv("prediction_2nd.csv")

In [11]:
exportCsv = X_validation_df[['département','circonscription','code','first_name','name','nuance_groupe','elu']]
exportCsv['code'] = exportCsv['code'].apply(str.lower)
exportCsv['code'] = exportCsv['code'].apply(str).apply(lambda x : x.replace('|', '-'))
exportCsv['candidat'] = exportCsv['first_name'].apply(str.capitalize) + ' ' + exportCsv['name'].apply(str.capitalize)
exportCsv['nom circo'] = exportCsv['département'].apply(
    lambda x : x.replace('-', ' ')).apply(str.title) + ' - ' + exportCsv['circonscription'].apply(str)
exportCsv = exportCsv.drop(['département', 'circonscription', 'first_name', 'name'], axis=1)
exportCsv = exportCsv.sort_values(['code', 'elu'], ascending=[True, False])

circos = exportCsv['code'].unique()
temp = pd.DataFrame()
d = {}
for c in circos:
    temp = exportCsv[exportCsv['code'] == c]
    l = []
    for index, rows in temp.iterrows():
        l.extend(rows[1:].values)
    d[c] = l
  
duels = {k: v for k, v in d.items() if len(v) < 9}

duelsDf = pd.DataFrame.from_dict(duels, orient='index')


    
duelsDf.columns = ['color1','score1','candidat1','nom circo','color2','score2','candidat2','nom circo']
duelsDf = duelsDf.iloc[:,:7]
print(duelsDf.head())

triangulaires = {k: v for k, v in d.items() if len(v) > 8}
triangulaires = pd.DataFrame(triangulaires)

triangulaires.index = ['color1','score1','candidat1','nom circo','color2','score2','candidat2','nom circo','color3','score3','candidat3','nom circo']
triangulaires = triangulaires.T

nomCirco = triangulaires['nom circo'].iloc[:,0]
triangulaires = triangulaires.drop('nom circo', axis=1)
triangulaires['nom circo'] = nomCirco
final = pd.concat([duelsDf, triangulaires])
final = final.fillna(value='')
final.index.name = 'circo'
print(final.sort_index().head())
final.to_csv('resultats2.csv', index=True)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

     color1  score1            candidat1               nom circo color2  \
25-4    REM       1     Frederic Barbier             Doubs - 4.0     FN   
91-4    REM       1  Marie-pierre Rixain           Essonne - 4.0     LR   
34-1    ECO       1  Jean-louis Roumegas           Herault - 1.0    REM   
42-4     LR       1         Dino Cinieri             Loire - 4.0    REM   
13-3    REM       1      Alexandra Louis  Bouches Du Rhone - 3.0     FN   

      score2          candidat2  
25-4     0.0      Sophie Montel  
91-4     0.0        Agnes Evren  
34-1     0.0  Patricia Miralles  
42-4     0.0      David Kauffer  
13-3     0.0    Stephane Ravier  
                       candidat1                 candidat2 candidat3 color1  \
circo                                                                         
1-1                Xavier Breton            Laurent Mallet               LR   
1-2    Charles De la verpilliere       Marie-jeanne Beguet               LR   
1-3                Olga Giver